In [5]:
import os
import pandas as pd

dfs = []
files = []
folder_path = "./buffer/sampes_v1"
files_lis = os.listdir(folder_path)

for file in files_lis:
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path, engine="openpyxl")  # 读取文件
        #print(f"读取文件: {file}")

        # ----
        # df['英文评论'] = ''
        # df['中文评论'] = ''
        # df['分析结果'] = ''
        # df['中文分析'] = ''
        # df['提取结果'] = ''

        dfs.append(df)
        files.append(file)

df = pd.concat(dfs, ignore_index=True)
print(df.columns)
print(df.shape)

df.to_csv('./docs/amz_reviews_part1.csv', index=False, sep='\t')

Index(['ASIN', '标题', '标题(翻译)', '内容', '内容(翻译)', 'VP评论', 'Vine Voice评论', '型号',
       '星级', '赞同数', '图片数量', '图片地址', '是否有视频', '视频地址', '评论链接', '评论人', '头像地址',
       '所属国家', '评论人主页', '红人计划链接', '评论时间'],
      dtype='object')
(803, 21)
